In [ ]:
!pip install langchain
!pip install langchain_openai
!pip install langchain_community
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3

In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_PROJECT'] = userdata.get('LANGCHAIN_PROJECT')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
# create a basic chain using LLMChain

prompt = ChatPromptTemplate.from_template("Give me a report about {topic}")

output_parser = StrOutputParser()

chain = LLMChain(
    prompt=prompt,
    llm=llm,
    output_parser=output_parser
    )

result = chain.run("machine learning")

In [ ]:
print(result)

Machine learning is a rapidly growing field in the realm of artificial intelligence that focuses on developing algorithms and models that allow computers to learn from and make predictions or decisions based on data. This technology has the potential to revolutionize industries such as healthcare, finance, and transportation by enabling more accurate predictions, personalized recommendations, and automation of tasks.

One of the key advantages of machine learning is its ability to handle large and complex datasets that traditional statistical methods may struggle with. By using algorithms such as neural networks, decision trees, and support vector machines, machine learning models can identify patterns and relationships in data that humans may not be able to detect.

Machine learning has already been successfully applied in various real-world applications, such as image and speech recognition, natural language processing, and autonomous vehicles. For example, companies like Google and 

In [ ]:
# LCEL implementation

lcel_chain = prompt | llm | output_parser

result = lcel_chain.invoke({"topic":"machine learning"})

In [ ]:
result

'Machine learning is a rapidly growing field in the field of artificial intelligence that focuses on developing algorithms and models that allow computers to learn from and make predictions or decisions based on data. Machine learning algorithms can be broadly categorized into supervised learning, unsupervised learning, and reinforcement learning.\n\nSupervised learning involves training a model on a labeled dataset, where the input data is paired with the correct output. The model learns to make predictions by finding patterns in the data and adjusting its parameters to minimize errors. Common supervised learning algorithms include linear regression, logistic regression, support vector machines, and neural networks.\n\nUnsupervised learning, on the other hand, involves training a model on an unlabeled dataset, where the model must find patterns or structure in the data without explicit guidance. Clustering algorithms, such as k-means and hierarchical clustering, and dimensionality red

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


def get_vector_store(embedding_function=OpenAIEmbeddings(), storage_directory="embeddings", collection_name="langchain"):
  vectordb = Chroma(persist_directory=storage_directory, embedding_function=embedding_function, collection_name=collection_name)
  return vectordb

In [ ]:
vectorstore_a = get_vector_store(collection_name="vectorstore_a")

In [ ]:
vectorstore_a.add_texts(['half of my text will be part of a', "VP's birthday is on 1st October"])

['f9a210fe-bd2b-4d84-b441-d97e70df43e5',
 '146ce094-0733-49b2-9c69-29a1fff3e926']

In [ ]:
vectorstore_b = get_vector_store(collection_name="vectorstore_b")


In [ ]:
vectorstore_b.add_texts(['half of the text is part of b', "VP was born in 1981 ", "VP is a friend of BP"])

['777c2f8a-8e30-49fa-ae69-5d3625ff22a9',
 '5c089a7f-160b-4dd7-b22d-9cbb9f8785bf',
 '02691df6-a9c6-4273-a59e-a067a0af2b89']

In [ ]:
vectorstore_a._collection, vectorstore_b._collection

(Collection(name=vectorstore_a), Collection(name=vectorstore_b))

In [ ]:
len(vectorstore_a.get()['ids']), len(vectorstore_b.get()['ids'])

(2, 5)

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
retriever_a = vectorstore_a.as_retriever()
retriever_b = vectorstore_b.as_retriever()

In [ ]:
prompt_str = """

Answer the following question basis the context that is provided.

Context : {context}

\n\n

Question : {question}

Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_str)



In [ ]:
retrieval = RunnableParallel(

                             {"context": retriever_a,
                              "question": RunnablePassthrough()}
)

chain = retrieval | prompt | llm | output_parser

In [ ]:
chain.invoke("when was VP born")



'VP was born on 1st October.'

In [ ]:
chain.invoke("give me the complete birthdate of VP")

"VP's complete birthdate is 1st October."

In [ ]:
prompt_str = """

Answer the following question basis the context that is provided.

Context :

{context_a}
\n\n
{context_b}
\n\n

Question : {question}

Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_str)



In [ ]:
retrieval_enhanced = RunnableParallel(
    {"context_a" : retriever_a,
     "context_b" : retriever_b,
     "question" : RunnablePassthrough()}
)

chain = retrieval_enhanced | prompt | llm | output_parser

In [ ]:
chain.invoke("when was VP born")

'VP was born in 1981.'

In [ ]:
chain.invoke("give me the complete birthdate of VP")

"VP's complete birthdate is 1st October, 1981."

In [ ]:
# Langchain abstraction that allow us to turn Python Functions into pipe compatible functions

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x + 5


def multiply_by_two(x):
    return x * 2



In [ ]:
multiply_by_two_lambda = RunnableLambda(multiply_by_two)

add_five_lambda = RunnableLambda(add_five)

In [ ]:
chain = multiply_by_two_lambda | add_five_lambda

In [ ]:
chain.invoke(10)

25

In [ ]:
def add_branding_info(x):
  print("Generative Geek Demo")
  print("\n\n")
  print("Report follows")
  print("\n")
  print(x)

add_branding_info_lambda = RunnableLambda(add_branding_info)

In [ ]:
prompt = ChatPromptTemplate.from_template("Give me a small report about {topic}")
lcel_chain = prompt | llm | output_parser

result = lcel_chain.invoke({"topic": "Market Mix Modeling"})
print(result)

Market Mix Modeling is a statistical analysis technique used by businesses to measure the impact of various marketing tactics on sales and other key performance indicators. It involves analyzing historical data to understand the effectiveness of different marketing strategies, such as advertising, promotions, pricing, and distribution channels.

By using Market Mix Modeling, companies can optimize their marketing budget allocation and make data-driven decisions to improve their overall marketing effectiveness. This technique helps businesses understand which marketing activities are driving sales and which are not, allowing them to reallocate resources to the most effective strategies.

Market Mix Modeling typically involves collecting and analyzing data on sales, marketing activities, and external factors such as economic conditions and competitor actions. This data is then used to build a statistical model that quantifies the impact of each marketing tactic on sales and other perform

In [ ]:
lcel_chain = prompt | llm | output_parser | add_branding_info_lambda
lcel_chain.invoke("Learning about langchain")


Generative Geek Demo



Report follows


Langchain is a decentralized language learning platform that utilizes blockchain technology to connect language learners with native speakers for personalized language exchange. Users can earn tokens by teaching their native language to others, which can then be used to pay for lessons in a language they want to learn.

The platform offers a variety of features such as language courses, live video lessons, and language exchange groups. Users can track their progress, set goals, and receive feedback from native speakers to improve their language skills.

Langchain aims to make language learning more accessible, affordable, and effective by leveraging the power of blockchain technology. It provides a unique opportunity for language learners to practice speaking with native speakers and immerse themselves in the language and culture they are trying to learn.

Overall, Langchain is a promising platform that has the potential to revolutionize the way